### Pattern Match

In [3]:
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

In [5]:
is_variable('?hello '), is_variable('?hello'), 

(False, True)

In [6]:
def pat_match_1(pattern, saying):
    if is_variable(pattern[0]):
        return True
    else:
        if pattern[0] != saying[0]:
            return False
        else:
            return pat_match_1(pattern[1:], saying[1:])

In [7]:
pat_match_1('I want ?X', "I want holiday")

True

In [9]:
pat_match_1('I have dreamed a ?X', "I dreamed about dog")

False

In [10]:
pat_match_1('I dreamed about ?X', "I dreamed about dog")

True

In [11]:
def pat_match_2(pattern, saying):
    if is_variable(pattern[0]):
        return pattern[0], saying[0]
    else:
        if pattern[0] != saying[0]: return False
        else:
            return pat_match_2(pattern[1:], saying[1:])

In [14]:
pattern = 'I want ?X'.split()
saying = "I want holiday".split()
pat_match_2(pattern, saying)

('?X', 'holiday')

In [17]:
pat_match_2("?X equals ?X".split(), "2+3 equals 2+2".split())

('?X', '2+3')

In [33]:
def pat_match(pattern, saying):
    if not pattern or not saying:
        return []
    if is_variable(pattern[0]):
        return [(pattern[0],saying[0])] + pat_match(pattern[1:], saying[1:])
    else:
        if pattern[0] != saying[0]:
            return []
        else:
            return pat_match(pattern[1:], saying[1:])

In [34]:
pat_match("?X greater than ?Y".split(), "3 greater than 2".split())

[('?X', '3'), ('?Y', '2')]

In [21]:
def pat_to_dict(patterns):
    return {k:v for k, v in patterns}

In [22]:
def subsitite(rule, parsed_rules):
    if not rule:
        return []
    return [parsed_rules.get(rule[0], rule[0])] + subsitite(rule[1:], parsed_rules)

In [23]:
got_patterns = pat_match("I want ?X".split(), "I want iPhone".split())
got_patterns

[('?X', 'iPhone')]

In [24]:
subsitite("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns))

['What', 'if', 'you', 'mean', 'if', 'you', 'got', 'a', 'iPhone']

In [28]:
' '.join(subsitite("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns)))

'What if you mean if you got a iPhone'

In [29]:
john_pat = pat_match('?P needs ?X'.split(), "John needs vacation".split())
john_pat

[('?P', 'John'), ('?X', 'vacation')]

In [31]:
' '.join(subsitite("Why does ?P need ?X ?".split(), pat_to_dict(john_pat)))

'Why does John need vacation ?'

***最简单的基于模板的对话生成:***

In [32]:
defined_patterns = {
    "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"], 
    "My ?X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"]
}

In [42]:
import random
def get_response(saying, rules=defined_patterns):
    """" please implement the code, to get the response as followings:
    >>> get_response('I need iPhone') 
    >>> Image you will get iPhone soon
    >>> get_response("My mother told me something")
    >>> Talk about more about your monther.
    """
    for pat, response_pats in rules.items():
        match = pat_match(pat.split(), saying.split())
        if match:
            match_dic = pat_to_dict(match)
            response_pat = random.choice(response_pats)
            return ' '.join(subsitite(response_pat.split(), match_dic))

In [58]:
get_response('I need iPhone')

'Why do you need iPhone ?'

In [68]:
get_response("My mother told me something")

'How do you think about your mother ?'

###  Segment Match

In [72]:
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

is_pattern_segment('?*P')

True

In [73]:
from collections import defaultdict

In [74]:
def segment_match(pattern, saying):
    """
    pattern: pattern[0] begin with '?*', eg:'?*P'
    """
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')
    if not rest:
        return (seg_pat, saying), len(saying)
    for i, token in enumerate(saying):
        if rest[0] == token and is_match(rest[1:], saying[(i+1):]):
            return (seg_pat, saying[:i]), i
    return (seg_pat, saying), len(saying)

def is_match(rest, saying):
    if not rest and not saying:
        return True
    if not all(a.isalpha() for a in rest[0]):
        return True
    if rest[0] != saying[0]:
        return False
    return is_match(rest[1:], saying[1:])

segment_match(['?*P', 'hello', 'world'], ['mike', 'jack', 'hello', 'world'])

(('?P', ['mike', 'jack']), 2)

In [134]:
 is_match(['hello', 'world'], ['hello'])

IndexError: list index out of range

In [76]:
fail = [True, None]

def pat_match_with_seg(pattern, saying):
    if not pattern or not saying:
        return []
    pat = pattern[0]
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg[pattern[1:], saying[1:]]
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail
        

In [77]:
segment_match('?*P is very good'.split(), "My dog and my cat is very good".split())

(('?P', ['My', 'dog', 'and', 'my', 'cat']), 5)

In [100]:
pat_match_with_seg('My ?*PL are very good ?*L'.split(), "My dog and my cat are very good and very friendly".split())

[('?PL', ['dog', 'and', 'my', 'cat']), ('?L', ['and', 'very', 'friendly'])]

In [105]:
segment_match('?*PL are good too'.split(), 'My dog are good'.split())

IndexError: list index out of range

In [79]:
response_pair = {
    'I need ?X': ["Why do you neeed ?X"],
    "I dont like my ?X": ["What bad things did ?X do for you?"]
}

In [80]:
pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())

[('?X', ['an', 'iPhone'])]

In [94]:
pat_match_with_seg('?*X hello ?*Y'.split(), "I am mike, hello ".split())

[('?X', ['I', 'am', 'mike,'])]

In [99]:
pat_match_with_seg('?*X need ?*P too'.split(), "I need an iPhone".split())

[('?X', ['I']), ('?P', ['an', 'iPhone'])]

In [81]:
subsitite("Why do you neeed ?X".split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())))

['Why', 'do', 'you', 'neeed', ['an', 'iPhone']]

In [82]:
def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}

In [83]:
subsitite("Why do you neeed ?X".split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())))

['Why', 'do', 'you', 'neeed', 'an iPhone']

***1.6 问题1***

In [110]:
rules = {
    "I was ?*X": ["Were you really ?X ?", "I already knew you were ?X ."],
    "?*X hello ?*Y": ["Hi, how do you do?"]
}

In [144]:
import random
def get_response(saying, response_rules=rules):
    responses = []
    for pat, response_pats in response_rules.items():
        match = pat_match_with_seg(pat.split(), saying.split())
        if match:
            match_dic = pat_to_dict(match)
            response_pat = random.choice(response_pats)
            responses.append(' '.join(subsitite(response_pat.split(), match_dic)))
    return responses

In [145]:
get_response("I was confused then")

['Were you really confused then ?', 'Hi, how do you do?']

In [146]:
rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?']}

sayings = """
hello hello world!
Now I want to fly.
I always konw what I want to do.
After the accident I was  so depressed.
After the vacation I feel  so good.
"""

for saying in sayings.strip().split('\n'):
    print('>>> ' + saying)
    print(get_response(saying, response_rules=rule_responses))
    print()

>>> hello hello world!
['Please state your problem', 'what would it mean if you got ?y', 'Do you really think its likely that ?y', 'why not?', 'Perhaps I already knew you were ?y', 'Do you often feel ?y ?']

>>> Now I want to fly.
['How do you do', 'Suppose you got to fly. soon', 'What do you think about ?y', "Are you saying 'No' just to be negative?", 'Perhaps I already knew you were ?y', 'What other feelings do you have?']

>>> I always konw what I want to do.
['How do you do', 'what would it mean if you got to do.', 'Do you wish that ?y', 'You are being a negative', 'Were you really', 'Do you often feel ?y ?']

>>> After the accident I was  so depressed.
['Please state your problem', 'what would it mean if you got ?y', 'Do you wish that ?y', 'You are being a negative', 'Were you really', 'What other feelings do you have?']

>>> After the vacation I feel  so good.
['How do you do', 'Suppose you got ?y soon', 'What do you think about ?y', "Are you saying 'No' just to be negative?", 'P